In [11]:
# Memoria GPU
import torch
#x = torch.rand(100,3,512,512).cuda() # simulacion de 100 imagenes de 512x512 a color...
print(torch.cuda.get_device_name(0))
print("Total Memory: ",round(torch.cuda.mem_get_info(0)[1]/1024**3,1),'GB')
print('Memory Usage:')
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')



NVIDIA GeForce RTX 3090
Total Memory:  23.7 GB
Memory Usage:
Allocated: 0.3 GB
Cached:    0.9 GB


Determinar de forma general un batch size apropiado para nuestro modelo.

https://discuss.pytorch.org/t/how-to-determine-the-largest-batch-size-of-a-given-model-saturating-the-gpu/146075

Aplicando una busqueda binaria con la funcion de Suraj.

En principio siempre se quiere conseguir el tamaño maximo de batch size que soporte el hardware.


In [15]:
import time
import torch

dimension_imagen = [3,32,32]

def proc_time(b_sz, model, n_iter=10):
    x = torch.rand(b_sz, *dimension_imagen).cuda()
    model.cuda()
    torch.cuda.synchronize()
    start = time.time()
    for _ in range(n_iter):
        model(x)
    torch.cuda.synchronize()
    end = time.time() - start
    throughput = b_sz * n_iter / end
    print(f"Batch: {b_sz} \t {throughput} samples/sec")
    return (b_sz, throughput, )


In [20]:
for i in range(1,51,5):
    proc_time(i,model)

Batch: 1 	 10919.822962770111 samples/sec
Batch: 6 	 20286.839177750906 samples/sec
Batch: 11 	 95305.39971080355 samples/sec
Batch: 16 	 161591.29304117506 samples/sec
Batch: 21 	 242378.60209135938 samples/sec
Batch: 26 	 198132.09302325582 samples/sec
Batch: 31 	 234234.2352729238 samples/sec
Batch: 36 	 256663.17185109638 samples/sec
Batch: 41 	 185909.6908108108 samples/sec
Batch: 46 	 252173.55116978174 samples/sec


https://stackoverflow.com/questions/46654424/how-to-calculate-optimal-batch-size 

Use the summaries provided by pytorchsummary (pip install) or keras (builtin).

E.g.

from torchsummary import summary
summary(model)
.....
.....
================================================================
Total params: 1,127,495
Trainable params: 1,127,495
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 13.93
Params size (MB): 4.30
Estimated Total Size (MB): 18.25
----------------------------------------------------------------
Each instance you put in the batch will require a full forward/backward pass in memory, your model you only need once. People seem to prefer batch sizes of powers of two, probably because of automatic layout optimization on the gpu.

Don't forget to linearly increase your learning rate when increasing the batch size.

Let's assume we have a Tesla P100 at hand with 16 GB memory.

(16000 - model_size) / (forward_back_ward_size)
(16000 - 4.3) / 18.25 = 1148.29
rounded to powers of 2 results in batch size 1024

In [18]:
from torchinfo import summary
import torch.nn as nn

model = nn.Sequential(
          nn.Conv2d(3,20,3),
          nn.ReLU(),
          nn.Conv2d(20,64,3),
          nn.ReLU()
        )
batch_size = 10
resumen = summary(model, input_size=(batch_size, 3, 32, 32))
print(resumen)

import re
model_size = re.search(r"Params size (MB):",resumen.__repr__())
print(model_size)


Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [10, 64, 28, 28]          --
├─Conv2d: 1-1                            [10, 20, 30, 30]          560
├─ReLU: 1-2                              [10, 20, 30, 30]          --
├─Conv2d: 1-3                            [10, 64, 28, 28]          11,584
├─ReLU: 1-4                              [10, 64, 28, 28]          --
Total params: 12,144
Trainable params: 12,144
Non-trainable params: 0
Total mult-adds (M): 95.86
Input size (MB): 0.12
Forward/backward pass size (MB): 5.45
Params size (MB): 0.05
Estimated Total Size (MB): 5.63
None


MONITORIZAR MEMORIA DE LA GPU EN ENTRENAMIENTOS
https://wandb.ai/wandb/common-ml-errors/reports/How-To-Use-GPU-with-PyTorch---VmlldzozMzAxMDk 